In [1]:
import numpy as np
import pandas as pd
import os, os.path

In [2]:
vocTrain = pd.read_stata("../0_RawData/SC5_spVocTrain_D_11-0-0.dta", convert_categoricals=False)
CATI = pd.read_stata("../0_RawData/SC5_pTargetCATI_D_11-0-0.dta", convert_categoricals=False)

In [3]:
variables = [
    "ID_t", # target id 
    "wave", # wave of episode
    "spell", # spell id
    #"subspell", # 0 for full episode, 1,2,3 for parts
    "ts15221_v1", # angestrebter Ausbildungsabschluss
    "ts15219_v1", # Ausbildungsabschluss: B.Sc, M.Sc, Ausbildung, etc.
    "ts15265", # Note des Abschluss
    "tg24103", # Episodenmodus
    "ts15201", # Ausbildungstyp
    "ts15218", # Erfolgreicher Abschluss
    "tg24159", # Fachwechsel gegenüber Vorepisode; 1=same, 2=diff
    "tg24121", # Hochschulwechsel gegenüber Vorepisode; 1=same, 2=diff
    "ts1512c", # Andauern der Episode
#     "tg2419a_w1", # Status in Unterbrechung
#     "tg2419b_w1", # ""
#     "tg2419c_w1"  # ""
    "ts1511m_g1", # Prüfmodul: Startdatum (Monat, ediert)
    "ts1511y_g1", # Prüfmodul: Startdatum (Jahr, ediert)
    "ts1512m_g1", # Prüfmodul: Enddatum (Monat, ediert)
    "ts1512y_g1", # Prüfmodul: Enddatum (Jahr, ediert)
]
variables_rename = {
    "ID_t" : "stud_ID",
    "ts15221_v1" : "pursued_degree",
    "ts15219_v1" : "achieved_degree",
    "ts15265" : "achieved_grade",
    "tg24103" : "episodemode",
    "ts15201" : "type_of_education",
    "ts15218" : "successfull_graduation",
    "tg24159" : "change_of_field",
    "tg24121" : "change_of_institution",
    "ts1512c" : "current_episode",
    "ts1511m_g1" : "start_month",
    "ts1511y_g1" : "start_year",
    "ts1512m_g1" : "end_month",
    "ts1512y_g1" : "end_year"
}
variables_rename_reverse = {v : k for k, v in variables_rename.items()}

In [4]:
subset = vocTrain[(vocTrain["subspell"] == 0) & (vocTrain.disagint != 1)][variables].rename(columns=variables_rename)

In [5]:
relevant_education = [
    10, # Studium an einer Universität, auch pädagogische Hochschule, Kunst- und Musikhochschule
    9, # Studium an einer Fachhochschule, auch Hochschule für angewandte Wissenschaften oder University of Applied Sciences genannt (nicht Verwaltungsfachhochschule
    -28 # Wert aus Rekrutierung pTargetCATI ## might be relevant
]
relevant_courses_of_study = [
    13, # BA ohne Lehramt
    #17, # Erstes Staatsexamen Lehramt
    8, # BA
    12, # BA Lehramt
    #-54, # Designbedingt fehlend
]
curr = subset[
    subset.type_of_education.isin(relevant_education) & # filter for uni/fh only
    subset.pursued_degree.isin(relevant_courses_of_study) # filter bachelor only
]
curr.successfull_graduation.value_counts()

 1.0     6462
 2.0     3102
-98.0      12
Name: successfull_graduation, dtype: int64

In [6]:
CATI.tg02001.value_counts() # 1 BA Lehramt, 3 BA

-54    54540
 3     10854
 2      3671
 1      1883
 4      1428
 8        45
 5        19
 7         7
 6         3
Name: tg02001, dtype: int64

### Joining Information and Considering Dropped Cases
Since the first wave information on intended degree is not in spVocTrain, but only in pCATI, we will join that one column into our dataframe.

#### Dropped Students
NEPS considers a change of university dropping out, which I do not. This will needed to be filtered for. I also need to find a way to clean and rebuild the data in a concise one-row format, instead of the episodes being spread out over multiple.

##### Figuring out
Things to look out for:
* Weird disagreeing spells
* No sucessfull graduation (2)
    * With no spell afterwards
    * With a spell afterwards, but a change of subject

##### To be considered
Dropped out, if ts15218 "succesfull graduation" has a value of 2 "no", spells of interest are students doing some bachelor degree at a university "Universität" or school of applied science "Fachhochschule".

Cases:
* dropped out -> no further spell of interest ✓
* dropped out -> spell of interest
    * -> changed subject, irrelevant of change of institution ✓
    * -> changed institution, but not subject ☓

In [7]:
# prepare subset of CATI for joining
other = CATI[
    CATI.wave == 1 & # the question was only asked in wave 1, and this way ID_t becomes the unique identifier
    CATI.tg02001.isin([1,3]) # filter for 3=BA and 1=BA Lehramt only
][["tg02001", "ID_t"]] # ID_t to join on, tg02001 holds the pursued degree

# join CATI subset into dataframe for more complete sample
curr = curr.join(other.set_index("ID_t"), on="stud_ID").rename(columns={"tg02001" : "CATI_pursued_degree"}) # integrate tg02001 into df to fill in ts15201 -28
# filter out those rows that CATI added nothing too, meaning those without information on pursued degree even after joining
curr = curr[curr.type_of_education != -28 | (curr.type_of_education == -28 & ~curr.CATI_pursued_degree.isnull())]

# reorder columns in tmp dataframe for a nicer overview, moving joined CATI degree next to vocTrain degree
cols = list(curr.columns)
purs_deg_idx = cols.index(variables_rename["ts15221_v1"]) + 1
cols = cols[:purs_deg_idx] + cols[-1:] + cols[purs_deg_idx:-1]
curr = curr[cols]

In [8]:
# filter out all students who have one spell with an unsuccesfull graduation
dropped_students = curr[(curr.successfull_graduation == 2)].stud_ID.unique()
dropped = curr[curr.stud_ID.isin(dropped_students)]

# check how many samples we have per condition
## dropped out -> no further spell of interest
case1 = dropped[
    (dropped.successfull_graduation == 2) &
    (dropped.stud_ID.isin(dropped.stud_ID.value_counts()[dropped.stud_ID.value_counts() == 1].index))
].shape[0]

## dropped out -> spell of interest
### -> changed subject, irrelevant of change of institution
# find all those who failed and have any preluding or subsequent spell
tmp = dropped[
    (dropped.successfull_graduation == 2) &
    (dropped.stud_ID.isin(dropped.stud_ID.value_counts()[dropped.stud_ID.value_counts() > 1].index))
]
# shift up all rows by one, then select those indices we filtered above
# this gives us the possibility to compare two subsequent rows using the same index
tmp2 = dropped.shift(-1).loc[tmp.index]
# We only want to compare rows for the same student, and then filter those who actually changed subject
# leaving out those who only switched institutions or had other reasons for ending their spell
# and also leaving out possible mismatches, where all recorded spells of interest were unsuccesfull,
# and thus the following row is already for the next student
tmp2 = tmp2[
    (tmp2.stud_ID == tmp.stud_ID) & 
    (tmp2.change_of_field == 2)
]
case2 = tmp2.shape[0]

# control
control = dropped[dropped.successfull_graduation == 2].shape[0]
# succesfull grads
succ = curr[curr.successfull_graduation == 1].shape[0]

# and readable output
width = 50
print(
    f"{'No further spell of interest':.<{width}}{case1:5}",
    f"{'Change of field':.<{width}}{case2:5}",
    f"{'Total of useful cases':.<{width}}{case1 + case2:5}",
    "",
    f"{'Total spells with unsuccesfull graduation':.<{width}}{control:5}",
    f"{'Filtered out spells with no graduation:':.<{width}}{control - case1 - case2:5}",
    "",
    f"{'Total number of succesfull graduations':.<{width}}{succ:5}",
    sep="\n"
)

No further spell of interest...................... 1000
Change of field................................... 1228
Total of useful cases............................. 2228

Total spells with unsuccesfull graduation......... 3102
Filtered out spells with no graduation:...........  874

Total number of succesfull graduations............ 6462


In [9]:
### The rest
curr['tmp'] = curr.stud_ID.astype(np.uint32).astype(str) + "_" + curr.wave.astype(np.uint32).astype(str) + "_" + curr.spell.astype(np.uint32).astype(str)
tmp.loc[tmp2.index]['tmp'] = (tmp.stud_ID.astype(np.uint32).astype(str) + "_" + tmp.wave.astype(np.uint32).astype(str) + "_" + tmp.spell.astype(np.uint32).astype(str)).loc[tmp2.index]

curr[
    curr.stud_ID.isin(
        curr[
            (curr.successfull_graduation == 2)
            & (dropped.stud_ID.isin(dropped.stud_ID.value_counts()[dropped.stud_ID.value_counts() > 1].index))
            & (~curr.tmp.isin(tmp.tmp))
        ].stud_ID.unique()
    )
]
df = pd.DataFrame([curr.tmp[curr.successfull_graduation==2], tmp.tmp]).T
df.columns=["curr","tmp"]
df[df.tmp == df.curr].curr.shape

AttributeError: 'DataFrame' object has no attribute 'tmp'